<div class='alert alert-info'>
    <h1 align="center">Course Scheduler</h1>
    <h3 align="center">Mohammad Rahdar</h3>    
</div>

Welcome to the **Course Scheduler App**! 🎓  

<u><a href="https://github.com/mo-rahdar/Course-Scheduler/" target="_blank">GitHub Link</a></u> 

- Upload your **Courses_info.xlsx** file using the button below.  
- Click **Run Scheduler** to generate an optimized course schedule.  
- The results will include:  
  - A saved file `Schedule_results.xlsx` with the schedule.  
  - Plots showing course assignments, conflicts, and available meeting times.  

👉 Once finished, you can download and optionally modify the schedule.  
If you make changes, use the **Schedule Checker App** (see Binder link in README) to validate your edits.


In [1]:
import traceback
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import Utils
import os

# Widgets
upload_input = widgets.FileUpload(accept='.xlsx', multiple=False, description='Upload Input Excel')
run_btn = widgets.Button(description='Run Scheduler', button_style='success')

upload_result = widgets.FileUpload(accept='.xlsx', multiple=False, description='Upload Results Excel')
check_btn = widgets.Button(description='Check Schedule', button_style='info')

out = widgets.Output()

# Display both workflows
display(widgets.HTML("<h3>1️⃣ Generate a Schedule</h3>"))
display(upload_input, run_btn)
display(widgets.HTML("<h3>2️⃣ Check a Modified Schedule</h3>"))
display(upload_result, check_btn)
display(out)


# callbacks
def on_run_clicked(b):
    with out:
        clear_output()
        if not upload_input.value:
            print("Please upload an Excel (.xlsx) file first.")
            return
        try:
            fname = Utils.save_uploaded_file(upload_input, mode='scheduler')
            print("Saved uploaded file as", fname)

            # Run your pipeline → return path of generated file
            result_file = Utils.run_pipeline(fname)

            print("\nGenerated file:", result_file)
            print("Exists:", os.path.exists(result_file))
            if os.path.exists(result_file):
                print("Size (bytes):", os.path.getsize(result_file))

            # Show in-page download link using base64 embedding
            Utils.create_download_link(result_file, label="⬇️ Download Schedule Results")

            # show files after run (helpful for debugging)
            print("\nFiles after run:", os.listdir())
  

            # print("🎉 Scheduling complete.")
            # display(HTML(f'<a href="{result_file}" download>⬇️ Download Schedule Results</a>'))

        except Exception as e:
            print("❌ Error during run:")
            traceback.print_exc()


def on_check_clicked(b):
    with out:
        clear_output()
        if not upload_result.value:
            print("Please upload a Schedule_results.xlsx file first.")
            return
        try:
            fname = Utils.save_uploaded_file(upload_result, mode='check')
            print("Saved uploaded results as", fname)

            # Run your check pipeline → return path of generated file
            result_file = Utils.run_check_pipeline(fname)

            
            print("✅ Check complete.")
            # display(HTML(f'<a href="{result_file}" download>⬇️ Download Check Report</a>'))

        except Exception as e:
            print("❌ Error during check:")
            traceback.print_exc()

# connect buttons
run_btn.on_click(on_run_clicked)
check_btn.on_click(on_check_clicked)


HTML(value='<h3>1️⃣ Generate a Schedule</h3>')

FileUpload(value=(), accept='.xlsx', description='Upload Input Excel')

Button(button_style='success', description='Run Scheduler', style=ButtonStyle())

HTML(value='<h3>2️⃣ Check a Modified Schedule</h3>')

FileUpload(value=(), accept='.xlsx', description='Upload Results Excel')

Button(button_style='info', description='Check Schedule', style=ButtonStyle())

Output()